In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

# 모델 및 토크나이저 로드
model_name = "unsloth/Meta-Llama-3.1-8B"  # 사용할 모델 이름
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-08-17 21:53:43.622715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 21:53:43.622763: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 21:53:43.623744: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-17 21:53:43.628790: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-17 21:53:44.398904: W tensorflow/compiler/tf2

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.677 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 101,
    use_rslora = False,
    loftq_config = None,
)


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from datasets import load_dataset, Dataset
from peft import PeftModel

In [4]:
# PDF 텍스트 추출 및 chunk 단위로 나누기
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    doc = fitz.open(file_path)
    text = ''
    for page in doc:
        text += page.get_text()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks

# FAISS DB 생성 함수
def create_vector_db(chunks, model_path="intfloat/multilingual-e5-base"):
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

# 경로 유니코드 정규화 함수
def normalize_path(path):
    return unicodedata.normalize('NFC', path)

# PDF 데이터프레임을 처리하여 DB 및 retriever 생성
def process_pdfs_from_dataframe(df, base_directory):
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 8})
        
        pdf_databases[pdf_title] = {
            'db': db,
            'retriever': retriever
        }
    return pdf_databases

# 질문과 관련된 텍스트 조각 찾기
def find_relevant_chunk(question, pdf_databases):
    """질문에 대해 가장 유사한 chunk 찾기"""
    relevant_chunk = None
    for pdf_title, data in pdf_databases.items():
        retriever = data['retriever']
        relevant_docs = retriever.get_relevant_documents(question)
        if relevant_docs:
            relevant_chunk = relevant_docs[0].page_content
            break  # 가장 유사한 chunk를 찾았으므로 반복 종료
    return relevant_chunk if relevant_chunk else ""

# Alpaca 포맷 정의
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# 모델 및 토크나이저 로드
EOS_TOKEN = tokenizer.eos_token

# 데이터 포맷팅 함수 정의
def formatting_prompts_func(examples, pdf_databases):
    questions = examples["Question"]
    answers = examples["Answer"]
    texts = []

    for question, answer in zip(questions, answers):
        # 질문에 가장 유사한 chunk를 찾음
        relevant_chunk = find_relevant_chunk(question, pdf_databases)
        if relevant_chunk:
            # Alpaca 포맷으로 변환
            text = alpaca_prompt.format(question, relevant_chunk, answer) + EOS_TOKEN
            texts.append(text)
        else:
            # 유사한 chunk가 없는 경우 기본 텍스트 형식 (옵션)
            text = alpaca_prompt.format(question, "", answer) + EOS_TOKEN
            texts.append(text)
    
    return {"text": texts}

# CSV 데이터셋 로드
csv_path = "/home/jovyan/temp/open/train.csv"  # CSV 파일 경로 설정
dataset = load_dataset('csv', data_files=csv_path, split='train')

# PDF 데이터 처리 및 DB, retriever 생성
base_directory = './'
csv_df = pd.read_csv(csv_path)
pdf_databases = process_pdfs_from_dataframe(csv_df, base_directory)

# 포맷팅 함수 적용
formatted_dataset = dataset.map(lambda x: formatting_prompts_func(x, pdf_databases), batched=True)

# 포맷팅된 데이터 확인
print(formatted_dataset[0])


Processing PDFs:   0%|                                                                                          | 0/16 [00:00<?, ?it/s]

Processing 1-1 2024 주요 재정통계 1권...


/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Processing PDFs:   6%|█████▏                                                                            | 1/16 [00:05<01:15,  5.01s/it]

Processing 2024 나라살림 예산개요...


Processing PDFs:  12%|██████████▎                                                                       | 2/16 [00:11<01:20,  5.76s/it]

...

Processing 월간 나라재정 2023년 12월호...


Processing PDFs: 100%|█████████████████████████████████████████████████████████████████████████████████| 16/16 [01:01<00:00,  3.87s/it]


Map:   0%|          | 0/496 [00:00<?, ? examples/s]

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


{'SAMPLE_ID': 'TRAIN_000', 'Source': '1-1 2024 주요 재정통계 1권', 'Source_path': './train_source/1-1 2024 주요 재정통계 1권.pdf', 'Question': '2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?', 'Answer': '2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?\n\n### Input:\n2. 재정수입\n3. 재정지출\n4. 지방재정 조정\n5. 총사업비 관리대상사업\n6. 계속비 대상사업\n주요 재정통계\n●\nⅠ.\n2\n01 재정체계\n▸중앙정부 재정체계는 예산(일반･특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 \n21개, 기금 68개로 구성\n∙2024년 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성\n∙2024년 기금 지출은 49개 사업성기금 81.2조원, 6개 사회보험성기금 107.1조원, 5개 계정성기금 \n30.1조원으로 구성\n[그림 1-1] 재정지출 구조(2024년 예산 총지출 기준)\n주: 괄호 안은 총계 기준 예산액을 의미\n자료: 디지털예산회계시스템\n2024 주요 재정통계 | 2024 Fiscal Statistics\nⅠ. 주요재정통계\nⅡ. 국제통계\n부록\nⅢ. 분야별 재정지출\nⅠ. 주요재정통계\n3\n▸예산은 ｢국가재정법｣에 근거해 정부가 편성하고 국회가 심의･의결로 확정한 

In [20]:
# 포맷팅된 데이터 확인
print(formatted_dataset[1])

{'SAMPLE_ID': 'TRAIN_001', 'Source': '1-1 2024 주요 재정통계 1권', 'Source_path': './train_source/1-1 2024 주요 재정통계 1권.pdf', 'Question': '2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?', 'Answer': '2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성되어 있습니다.', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?\n\n### Input:\n2. 재정수입\n3. 재정지출\n4. 지방재정 조정\n5. 총사업비 관리대상사업\n6. 계속비 대상사업\n주요 재정통계\n●\nⅠ.\n2\n01 재정체계\n▸중앙정부 재정체계는 예산(일반･특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 \n21개, 기금 68개로 구성\n∙2024년 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성\n∙2024년 기금 지출은 49개 사업성기금 81.2조원, 6개 사회보험성기금 107.1조원, 5개 계정성기금 \n30.1조원으로 구성\n[그림 1-1] 재정지출 구조(2024년 예산 총지출 기준)\n주: 괄호 안은 총계 기준 예산액을 의미\n자료: 디지털예산회계시스템\n2024 주요 재정통계 | 2024 Fiscal Statistics\nⅠ. 주요재정통계\nⅡ. 국제통계\n부록\nⅢ. 분야별 재정지출\nⅠ. 주요재정통계\n3\n▸예산은 ｢국가재정법｣에 근거해 정부가 편성하고 국회가 심의･의결로 확정한 재정지출계획을 의미하며, \n일반회계와 특별

In [13]:
from transformers import TrainingArguments


training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=60,
    learning_rate=2e-4,
    fp16=True,  # 또는 bf16=True, 또는 둘 다 False로 설정
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=101,
    output_dir="outputs",
)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

Map (num_proc=2):   0%|          | 0/496 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [14]:
# 훈련 실행
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 496 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.762800
2,2.191100
3,2.040700
4,2.212600
5,2.096400
6,2.074400
7,1.882900
8,1.707100
9,1.631800
10,1.902400


In [23]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "기금과 예산의 다른점은?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n기금과 예산의 다른점은?\n\n### Input:\n\n\n### Response:\n기금은 재정수지와 무관하며, 재정수지에 직접적으로 영향을 미치지 않는다.<|end_of_text|>']

In [ ]:
#2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.

In [ ]:
#model.save_pretrained("llama3_model_거의최종") # Local saving
#tokenizer.save_pretrained("llama3_model_거의최종")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving